# Research Question: 
### Are the Win/Loss Ratio's of Pro Sports teams Correlated to their City's population?

### This notebook will go through CSV and .html files, cleaning / wrangling data to ultimately perform statistical tests, groupbys, and merges, to answer the above question.
#### -- By John G. Crowley, for University of Michigan's final project for Applied Data Science

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#Read in NHL
nhl_df=pd.read_csv("assets/nhl.csv")

#Read in Cities
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#Clean Cities Column name Population
pop = cities.columns[1]
cities.rename(columns = {pop:'Population'},inplace=True)

#NHL Preparation
nhl = nhl_df[nhl_df['year']==2018].drop(nhl_df.index[[0,9,18,26]])
nhl['ratio'] = nhl['W'].astype(float) / (nhl['W'].astype(float) + nhl['L'].astype(float))

#Regex to Clean NHL City Titles
nhl['team'].replace(to_replace='\*$',value='',regex=True,inplace=True)

#Regex to Clean off the Team name (most of it except few cases), then stripping off white space
nhl['Metropolitan area'] = nhl['team'].astype(str).str.replace('[A-Za-z]+$','').str.strip()

#Pass in manual dictionary of updates for Teams that only had 2 strings, 
#metro area or had different city name than
updates = {"Florida":"Miami-Fort Lauderdale",
           "Carolina":"Raleigh",
           "Vegas Golden":"Las Vegas",
           "Anaheim":"Los Angeles",
           "Arizona":"Phoenix",
          "New Jersey":"New York",
          "Minnesota":"Minneapolis-Saint Paul",
          "Colorado":"Denver"}

nhl['Metropolitan area'] = nhl['Metropolitan area'].replace(updates,regex=True)

#Nested loop through both cities and nhl dataframes to see if there is a match of city name, 
#where there is a match, replace NHL with the metropolitan area to get ready for the merge
for city in nhl['Metropolitan area']:
        for metroarea in cities['Metropolitan area']:
            if city.split()[0][:4] in metroarea:
                nhl['Metropolitan area'].replace(to_replace=city,value=metroarea,inplace=True)

nhl.at[29,'Metropolitan area'] = 'San Francisco Bay Area'      
        
#Merge Cities and NHL dfs    
def merging_team_df(df_teams,df_cities):
    
    merged = pd.merge(df_teams[['team','ratio','Metropolitan area']], df_cities[['Population','Metropolitan area']])
    new_df = merged.groupby('Metropolitan area').agg({'team':'unique','Population':'first','ratio':'mean'}).reset_index()
    
    return new_df

def nhl_correlation(): 
    
    new_df = merging_team_df(nhl,cities)

    population_by_region = new_df['Population'].astype(int) # pass in metropolitan area population from cities
    win_loss_by_region = new_df['ratio']
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()


<ipython-input-1-2ea80a94cf13>:25: FutureWarning: The default value of regex will change from True to False in a future version.
  nhl['Metropolitan area'] = nhl['team'].astype(str).str.replace('[A-Za-z]+$','').str.strip()


0.012486162921209923

In [2]:
nhl.reset_index()

,index,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,ratio,Metropolitan area
0,1,Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,0.701299,Tampa Bay Area
1,2,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,0.714286,Boston
2,3,Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,0.653333,Toronto
3,4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,0.594595,Miami–Fort Lauderdale
4,5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL,0.434783,Detroit
5,6,Montreal Canadiens,82,29,40,13,71,.433,209,264,-0.68,0.00,.378,27,2018,NHL,0.420290,Montreal
6,7,Ottawa Senators,82,28,43,11,67,.409,221,291,-0.85,0.00,.372,26,2018,NHL,0.394366,Ottawa
7,8,Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL,0.357143,Buffalo
8,10,Washington Capitals,82,49,26,7,105,.640,259,239,0.21,-0.04,.585,46,2018,NHL,0.653333,"Washington, D.C."
9,11,Pittsburgh Penguins,82,47,29,6,100,.610,272,250,0.23,-0.04,.573,45,2018,NHL,0.618421,Pittsburgh


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#Read in NBA.csv
nba_df=pd.read_csv("assets/nba.csv")

#Read in Metro Area.html
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#Clean Cities Column name Population
pop = cities.columns[1]
cities.rename(columns = {pop:'Population'},inplace=True)

#Clean up NBA team name from asteriks and parenthesis after Team Names
#Filter to year 2018

nba = nba_df[nba_df['year']==2018].copy()
nba['team'] = nba['team'].str.extract('([A-Za-z0-9\s]+)')
nba['Metropolitan area'] = nba['team']

#Change Name of W/L% to 'Percentage'
percentage = nba.columns[3]
nba.rename(columns = {percentage:'ratio'},inplace=True)

#Cast 'Ratio' column to Float type for Aggregation
nba['ratio'] = nba['W'].astype(float) / (nba['W'].astype(float) + nba['L'].astype(float))

#Groom Data in new Metro Column which will be merged on
nba['Metropolitan area'] = nba['Metropolitan area'].str.strip()

#This matches most items, save New Oreleans and Golden State / Utah 
for city in nba['Metropolitan area']:
        for metroarea in cities['Metropolitan area']:
            if city.split()[0][:4] in metroarea:
                nba['Metropolitan area'].replace(to_replace=city,value=metroarea,inplace=True)

nba['Metropolitan area'].iloc[20] = 'New Orleans'
nba['Metropolitan area'].iloc[21] = 'San Antonio'
                
Updates = {"Golden State Warriors":"San Francisco Bay Area",
"Brooklyn Nets":"New York City",
"Utah Jazz":"Salt Lake City"}

nba['Metropolitan area'] = nba['Metropolitan area'].replace(Updates,regex=True)

def nba_correlation():
    
    new_df = merging_team_df(nba,cities)

    population_by_region = new_df['Population'].astype(int) # pass in metropolitan area population from cities
    win_loss_by_region = new_df['ratio'].astype(float) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#Clean Cities Column name Population
pop = cities.columns[1]
cities.rename(columns = {pop:'Population'},inplace=True)

mlb = mlb_df[mlb_df['year'] == 2018]
mlb = mlb.drop(columns={'W-L%'}).copy()
mlb['ratio'] = (mlb['W'] / (mlb['W'] + mlb['L']).astype(float)).copy()

mlb['Metropolitan area'] = mlb['team'].copy()


updates = {'Arizona Diamondbacks':'Phoenix',
           'Texas Rangers':'Dallas-Fort Worth',
           'Colorado Rockies':'Denver'}

mlb['Metropolitan area'].replace(updates,regex=True,inplace=True)

for city in mlb['Metropolitan area']:
        for metroarea in cities['Metropolitan area']:
            if city.split()[0][:4] in metroarea:
                mlb['Metropolitan area'].replace(to_replace=city,value=metroarea,inplace=True)

mlb.at[29,'Metropolitan area'] = 'San Diego'
mlb.at[11,'Metropolitan area'] = 'San Francisco Bay Area'
 
def mlb_correlation(): 
    
    new_df = merging_team_df(mlb,cities)
    
    population_by_region = new_df['Population'].astype(int) # pass in metropolitan area population from cities
    win_loss_by_region = new_df['ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#Clean Cities Column name Population
pop = cities.columns[1]
cities.rename(columns = {pop:'Population'},inplace=True)

nfl = nfl_df[nfl_df['year'] == 2018].drop(nfl_df.index[[0,5,10,15,20,25,30,35]]).copy()
nfl = nfl.reset_index().drop(columns='index').copy()
nfl.rename(columns={'W-L%':'ratio'},inplace=True)
nfl['ratio'] = (nfl['W'].astype(float) / (nfl['W'].astype(float) + nfl['L'].astype(float))).copy()
nfl['team'] = nfl['team'].str.extract('([A-Za-z0-9\s]+)')
nfl['Metropolitan area'] = nfl['team']

for city in nfl['Metropolitan area']:
        for metroarea in cities['Metropolitan area']:
            if city.split()[0][:4] in metroarea:
                nfl['Metropolitan area'].replace(to_replace=city,value=metroarea,inplace=True)

nfl.at[24,'Metropolitan area'] = 'New Orleans'
nfl.at[0,'Metropolitan area'] = 'Boston'
nfl.at[10,'Metropolitan area'] = 'Nashville'
nfl.at[15,'Metropolitan area'] = 'San Francisco Bay Area'
nfl.at[25,'Metropolitan area'] = 'Charlotte'
nfl.at[31,'Metropolitan area'] = 'Phoenix'


def nfl_correlation(): 
    new_df = merging_team_df(nfl,cities)
    
    population_by_region = new_df['Population'].astype(int) # pass in metropolitan area population from cities
    win_loss_by_region = new_df['ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

0.004922112149349409

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from scipy.stats import ttest_rel
import math

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#Clean Cities Column name Population
pop = cities.columns[1]
cities.rename(columns = {pop:'Population'},inplace=True)

mlb_df = merging_team_df(mlb,cities)
nhl_df = merging_team_df(nhl,cities)
nba_df = merging_team_df(nba,cities)
nfl_df = merging_team_df(nfl,cities)

mlb_df.rename(columns={'team':'MLB Team','ratio':'MLB ratio'},inplace=True)
nhl_df.rename(columns={'team':'NHL Team','ratio':'NHL ratio'},inplace=True)
nba_df.rename(columns={'team':'NBA Team','ratio':'NBA ratio'},inplace=True)
nfl_df.rename(columns={'team':'NFL Team','ratio':'NFL ratio'},inplace=True)

#Merge MLB / NHL
df = mlb_df.merge(nhl_df,on=['Metropolitan area','Population'])
MLB_NHL = df[['Metropolitan area', 'Population','MLB Team','MLB ratio','NHL Team','NHL ratio']]

#Merge MLB / NBA
df2 = mlb_df.merge(nba_df,on=['Metropolitan area','Population'])
MLB_NBA = df2[['Metropolitan area', 'Population','MLB Team','MLB ratio','NBA Team','NBA ratio']]

#Merge MLB / NFL
df3 = mlb_df.merge(nfl_df,on=['Metropolitan area','Population'])
MLB_NFL = df3[['Metropolitan area', 'Population','MLB Team','MLB ratio','NFL Team','NFL ratio']]

#Merge NHL / NBA
df4 = nhl_df.merge(nba_df,on=['Metropolitan area','Population'])
NHL_NBA = df4[['Metropolitan area', 'Population','NHL Team','NHL ratio','NBA Team','NBA ratio']]

#Merge NHL / NFL
df5 = nhl_df.merge(nfl_df,on=['Metropolitan area','Population'])
NHL_NFL = df5[['Metropolitan area', 'Population','NHL Team','NHL ratio','NFL Team','NFL ratio']]

#Merge NFL / NBA
df6 = nfl_df.merge(nba_df,on=['Metropolitan area','Population'])
NFL_NBA = df6[['Metropolitan area', 'Population','NFL Team','NFL ratio','NBA Team','NBA ratio']]

def sports_team_performance():
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    dataframes = [NHL_NFL, NFL_NBA, NHL_NBA, MLB_NBA, MLB_NHL, MLB_NFL]
    dataframe_strings = ['NHL_NFL', 'NFL_NBA', 'NHL_NBA', 'MLB_NBA', 'MLB_NHL', 'MLB_NFL']

    df_dict = dict(zip(dataframe_strings,[d for d in range(6)]))
    
    for i in p_values:
        for j in p_values:
            if i == j:
                continue
            else:
                df_name = i+'_'+j
                for k,v in df_dict.items():
                    if k == df_name:
                        df = dataframes[v]
                        rat1 = i+' '+'ratio'
                        rat2 = j+' '+'ratio'
                        p_values[i][j] = ttest_rel(df[rat1],df[rat2])[1]
                        p_values[j][i] = ttest_rel(df[rat2],df[rat1])[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
